In [2]:
!pip install pyarrow<15.0.0a0,>=14.0.1
!pip install -q bitsandbytes
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install -q git+https://github.com/huggingface/peft.git
!pip install -q git+https://github.com/huggingface/accelerate.git
!pip install datasets scipy ipywidgets evaluate
!pip install requests==2.31.0
!pip install -q wandb
!pip install rouge_score

/bin/bash: line 1: 15.0.0a0,: No such file or directory
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 6.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.3 

In [3]:
import numpy as np
import wandb
import os
import re
import nltk
import string
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, PeftModel
from huggingface_hub import notebook_login
import transformers
from tqdm import tqdm
from accelerate import Accelerator
from datetime import datetime
from datasets import load_dataset, load_metric
import evaluate
import warnings
rouge = evaluate.load('rouge')
warnings.filterwarnings('ignore')
nltk.download('punkt')

wandb.login()
notebook_login()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [5]:
dataset = load_dataset('json', data_files={'train': 'train_dataset.json',
                                           'validation': 'val_dataset.json'})
train_dataset = dataset['train']
eval_dataset = dataset['validation']
print(train_dataset)
print(eval_dataset)

Dataset({
    features: ['input', 'output'],
    num_rows: 21
})
Dataset({
    features: ['input', 'output'],
    num_rows: 7
})


In [6]:
base_model_id = "mistralai/Mistral-7B-v0.1"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [7]:
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    model_max_length=512,
    padding_side="left",
    add_eos_token=True)

tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

### Clean text, add prompt structure, tokenize dataset

In [9]:
def clean_text(text):
    sentences = nltk.sent_tokenize(text.strip())
    sentences_cleaned = [s for sent in sentences for s in sent.split("\n")]
    sentences_cleaned_no_titles = [sent for sent in sentences_cleaned if len(sent) > 0 and sent[-1] in string.punctuation]
    text_cleaned = "\n".join(sentences_cleaned_no_titles)
    return text_cleaned

def tokenize(prompt):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=512,
        padding="max_length",
    )
    result["labels"] = result["input_ids"].copy()
    return result

def generate_and_tokenize_prompt(data_point):
    full_prompt = "### Question: {input}\n### Answer: {output}".format(input=clean_text(data_point['input']), output=clean_text(data_point['output']))
    return tokenize(full_prompt)

tokenized_train_dataset = train_dataset.map(generate_and_tokenize_prompt)
tokenized_eval_dataset = eval_dataset.map(generate_and_tokenize_prompt)

print(tokenized_train_dataset)
print(tokenized_eval_dataset)

Map:   0%|          | 0/21 [00:00<?, ? examples/s]

Map:   0%|          | 0/7 [00:00<?, ? examples/s]

Dataset({
    features: ['input', 'output', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 21
})
Dataset({
    features: ['input', 'output', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 7
})


## Check initial metrics

In [11]:
candidates = []
references = []

for item in tqdm(tokenized_eval_dataset):
    eval_prompt = item['input']
    model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

    response = tokenizer.decode(model.generate(**model_input, max_new_tokens=512)[0],
                                skip_special_tokens=True,
                                pad_token_id=tokenizer.eos_token_id)
    candidates.append(response)
    references.append(item['output'])

results = rouge.compute(predictions=candidates, references=references)
print(results)

100%|██████████| 7/7 [04:59<00:00, 42.77s/it]


{'rouge1': 0.16377685882629334, 'rouge2': 0.046982411354296746, 'rougeL': 0.12755042749961515, 'rougeLsum': 0.12139481949590211}


In [9]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [10]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    bias="none",
    lora_dropout=0.05,  # Conventional
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

# Apply the accelerator. You can comment this out to remove the accelerator.
accelerator = Accelerator()
model = accelerator.prepare_model(model)

trainable params: 21260288 || all params: 3773331456 || trainable%: 0.5634354746703705


In [11]:
wandb_project = "mistral-gutenberg-books-finetune"
if len(wandb_project) > 0:
    os.environ["WANDB_PROJECT"] = wandb_project

In [12]:
if torch.cuda.device_count() > 1: # If more than 1 GPU
    model.is_parallelizable = True
    model.model_parallel = True

# model.gradient_checkpointing_enable()

In [16]:
project = "gutenberg-books-finetune"
base_model_name = "mistral"
run_name = base_model_name + "-" + project
output_dir = "./" + run_name

tokenizer.pad_token = tokenizer.eos_token

args = transformers.TrainingArguments(
        output_dir=output_dir,
        warmup_steps=5,
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        gradient_checkpointing=True,
        gradient_accumulation_steps=4,
        max_steps=100,
        learning_rate=2.5e-5, # Want about 10x smaller than the Mistral learning rate
        logging_steps=10,
        bf16=True,
        metric_for_best_model="rouge1",
        load_best_model_at_end=True,
        optim="paged_adamw_8bit",
        logging_dir="./logs",        # Directory for storing logs
        save_strategy="steps",       # Save the model checkpoint every logging step
        save_steps=10,                # Save checkpoints every 50 steps
        evaluation_strategy="steps", # Evaluate the model every logging step
        eval_steps=10,               # Evaluate and save checkpoints every 50 steps
        do_eval=True,                # Perform evaluation at the end of training
        report_to="wandb",           # Comment this out if you don't want to use weights & baises
        run_name=f"{run_name}-{datetime.now().strftime('%Y-%m-%d-%H-%M')}"          # Name of the W&B run (optional)
    )

data_collator = transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [17]:
metric = load_metric("rouge", trust_remote_code=True)

def preprocess_logits_for_metrics(logits, labels):
    # the actualy prediction is the first element of the tuple
    if isinstance(logits, tuple):
        logits = logits[0]
    return logits.argmax(dim=-1)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip()))
                      for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip()))
                      for label in decoded_labels]

    # Compute ROUGE scores
    result = metric.compute(predictions=decoded_preds, references=decoded_labels,
                            use_stemmer=True)

    # Extract ROUGE f1 scores
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    # Add mean generated length to metrics
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id)
                      for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [18]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
    compute_metrics=compute_metrics,
    args=args,
    data_collator=data_collator,
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,
)

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
10,2.138800,1.958890,51.038700,19.848700,44.657100,49.208400,512.000000
20,1.774500,1.711860,54.777600,23.708600,49.538500,53.298300,512.000000
30,1.428300,1.553371,55.933100,25.086600,50.577100,54.672000,511.857100
40,1.113900,1.480875,56.834700,25.783800,51.017500,55.496400,512.000000
50,0.820000,1.483367,58.510300,27.532700,52.631100,57.204200,512.000000
60,0.581400,1.566640,59.493300,30.251300,54.205300,58.392500,512.000000
70,0.371100,1.685652,60.163500,30.877200,54.777000,58.914300,512.000000
80,0.251900,1.815350,59.639700,30.000700,53.794700,58.322900,512.000000
90,0.166600,1.914930,59.217000,30.322500,53.536700,57.812100,512.000000
100,0.133500,1.963228,58.802000,29.531800,52.866200,57.425200,512.000000


TrainOutput(global_step=100, training_loss=0.8779978919029235, metrics={'train_runtime': 9288.0314, 'train_samples_per_second': 0.086, 'train_steps_per_second': 0.011, 'total_flos': 1.6738674196611072e+16, 'train_loss': 0.8779978919029235, 'epoch': 36.36363636363637})

Due to the model quite huge and dataset extremely low the overfitting take place. So, take 50th step as the best model

In [4]:
base_model_id = "mistralai/Mistral-7B-v0.1"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,  # Mistral, same as before
    quantization_config=bnb_config,  # Same quantization config as before
    device_map="auto",
    trust_remote_code=True,
)

eval_tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    add_bos_token=True,
    trust_remote_code=True,
)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [21]:
eval_prompt = "### Question: Make me a summary of 'Rivals of the Clouds' by Raoul Whitfield.\n### Answer:"

# Re-init the tokenizer so it doesn't add padding or eos token
ft_model = PeftModel.from_pretrained(base_model, "mistral-gutenberg-books-finetune/checkpoint-50")

eval_tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    add_bos_token=True,
)

model_input = eval_tokenizer(eval_prompt, return_tensors="pt").to("cuda")

ft_model.eval()
with torch.no_grad():
    response = eval_tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=512)[0], skip_special_tokens=True)
    try:
      print(re.search(rf'(?<=### Answer: )[^\n\n]*', response, flags=re.I).group(0))
    except:
      print(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Set during World War I, 'Rivals of the Clouds' by Raoul Whitfield follows the tense and turbulent experiences of Allied pilots Lieutenant Adams and newcomer Tex Langdon. Amid the perils of aerial combat and thick ground fog, the story explores themes of rivalry, redemption, and camaraderie as the two pilots navigate their distrust and eventual mutual respect in order to survive the perils of the sky. Ultimately, it's a story of men faced with the extreme dangers of war and the transformative power of shared adversity.


In [24]:
ft_model.push_to_hub("mistral-gutenberg-books-finetune")

adapter_model.safetensors:   0%|          | 0.00/347M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/NikyParfenov/mistral-gutenberg-books-finetune/commit/3dd43f848d57f4c8212b4b21613ef7327c2aecc0', commit_message='Upload model', commit_description='', oid='3dd43f848d57f4c8212b4b21613ef7327c2aecc0', pr_url=None, pr_revision=None, pr_num=None)

In [5]:
adapter_model = "NikyParfenov/mistral-gutenberg-books-finetune"
ft_model = PeftModel.from_pretrained(base_model, adapter_model)

adapter_config.json:   0%|          | 0.00/742 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/347M [00:00<?, ?B/s]

In [8]:
eval_prompt = "### Question: Make me a summary of 'Rivals of the Clouds' by Raoul Whitfield.\n### Answer:"
eval_tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    add_bos_token=True,
)

model_input = eval_tokenizer(eval_prompt, return_tensors="pt").to("cuda")

ft_model.eval()
with torch.no_grad():
    response = eval_tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=512)[0], skip_special_tokens=True)
    try:
      print(re.search(rf'(?<=### Answer: )[^\n\n]*', response, flags=re.I).group(0))
    except:
      print(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Set during World War I, 'Rivals of the Clouds' follows the tense relationship between pilots Lieutenant 'Tex' Langdon and Lieutenant 'Dizzy' Davis. After a series of mishaps and near-collisions, Tex is shot down and captured, while Dizzy becomes a national hero. The story explores themes of redemption and camaraderie as Tex escapes and joins a squadron in France, determined to prove himself and earn Dizzy's respect. Through harrowing aerial battles and the dangers of no man's land, Tex ultimately demonstrates his skill and bravery, leading to a reconciliation between the two pilots. Raoul Whitfield's vivid war narrative underscores the personal struggles and the high-stakes nature of aerial combat during the Great War.


## Let's add google search and compare results

In [12]:
import http.client
import json

def run_google_search(prompt):
    conn = http.client.HTTPSConnection("google.serper.dev")
    payload = json.dumps({
    "q": prompt,
    "page": 1,
    "num": 20
    })
    headers = {
    'X-API-KEY': os.environ['SERPER_KEY'],
    'Content-Type': 'application/json'
    }
    conn.request("POST", "/search", payload, headers)
    res = conn.getresponse()
    data = res.read()

    return str([f"TITLE: {item['title']}; SNIPPET: {item['snippet']}" for item in json.loads(data.decode("utf-8"))['organic']])

In [13]:
adapter_model = "NikyParfenov/mistral-gutenberg-books-finetune"
base_model_id = "mistralai/Mistral-7B-v0.1"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    add_bos_token=True,
)

ft_model = PeftModel.from_pretrained(base_model, adapter_model)
ft_model.generation_config.pad_token_id = tokenizer.pad_token_id

candidates = []
references = []

for item in tqdm(tokenized_eval_dataset):
    rag = run_google_search(item['input'])
    eval_prompt = "### {rag}\n### Question: {input}\n### Answer:".format(rag=rag, input=item['input'])
    model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

    response = tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=512)[0],
                                skip_special_tokens=True,
                                pad_token_id=tokenizer.eos_token_id)
    try:
        msg = re.search(rf'(?<=### Answer: )[^\n\n]*', response, flags=re.I).group(0)
    except:
        msg = response

    candidates.append(msg)
    references.append(item['output'])

results = rouge.compute(predictions=candidates, references=references)
print(results)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_config.json:   0%|          | 0.00/742 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/347M [00:00<?, ?B/s]

100%|██████████| 7/7 [07:44<00:00, 66.35s/it]


{'rouge1': 0.41674781502238, 'rouge2': 0.161282673305801, 'rougeL': 0.28228741048070916, 'rougeLsum': 0.28216720843692855}


The final result with google search is a bit worse, it seems a lot of noise has been found in web browsing